In [46]:
from transformers import AutoTokenizer , AutoModelForSequenceClassification

In [47]:
import pandas as pd
data = pd.read_csv("ChnSentiCorp_htl_all.csv")
data.head()

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"


In [48]:
data = data.dropna()

In [49]:
from torch.utils.data import Dataset

class MyDataSet(Dataset):
    
    def __init__(self):
        super().__init__()
        self.data = pd.read_csv("ChnSentiCorp_htl_all.csv")
        self.data = self.data.dropna()
    
    def __getitem__(self, index):
        return self.data.iloc[index]["review"] , self.data.iloc[index]["label"]
    
    def __len__(self):
        return len(self.data)
    
    



In [50]:
dataset = MyDataSet()
for i in range(5):
    print(dataset[i])

('距离川沙公路较近,但是公交指示不对,如果是"蔡陆线"的话,会非常麻烦.建议用别的路线.房间较为简单.', 1)
('商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!', 1)
('早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。', 1)
('宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小，但加上低价位因素，还是无超所值的；环境不错，就在小胡同内，安静整洁，暖气好足-_-||。。。呵还有一大优势就是从宾馆出发，步行不到十分钟就可以到梅兰芳故居等等，京味小胡同，北海距离好近呢。总之，不错。推荐给节约消费的自助游朋友~比较划算，附近特色小吃很多~', 1)
('CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风', 1)


In [51]:
from torch.utils.data import random_split
trainset, validset = random_split(dataset,[0.9,0.1])
len(trainset),len(validset)

(6989, 776)

In [52]:
import torch
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("hfl/rbt3")

def collate_func(batch):
    texts, labels = [],[]
    for item in batch:
        texts.append(item[0])
        labels.append(item[1])
    inputs = tokenizer(texts, max_length=128, padding="max_length", truncation=True, return_tensors="pt")
    inputs["labels"] = torch.tensor(labels)
    return inputs

In [53]:
from torch.utils.data import DataLoader

trainloader = DataLoader(trainset,batch_size = 32 , shuffle= True,collate_fn= collate_func)
validloader = DataLoader(validset , batch_size= 64 , shuffle= False, collate_fn= collate_func)



In [54]:
next(enumerate(trainloader))

(0,
 {'input_ids': tensor([[ 101, 1825, 3315,  ..., 1416, 8024,  102],
         [ 101, 6983, 2421,  ...,    0,    0,    0],
         [ 101, 8182,  118,  ..., 1091, 4638,  102],
         ...,
         [ 101, 2190, 6421,  ...,    0,    0,    0],
         [ 101, 1765, 4157,  ...,    0,    0,    0],
         [ 101, 3221, 2769,  ..., 5892, 1355,  102]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 1, 1, 1],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0,
         1, 0, 1, 1, 1, 0, 1, 0])})

In [55]:
next(enumerate(validloader))[1]

{'input_ids': tensor([[ 101, 3680, 3613,  ...,    0,    0,    0],
        [ 101, 2523, 1962,  ...,    0,    0,    0],
        [ 101, 1333, 1044,  ..., 3302, 1218,  102],
        ...,
        [ 101, 4696,  679,  ...,    0,    0,    0],
        [ 101, 3221, 5018,  ...,    0,    0,    0],
        [ 101, 5439, 4277,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
        0, 1, 0

In [56]:
model = AutoModelForSequenceClassification.from_pretrained("hfl/rbt3")
from torch.optim import Adam

optimizer = Adam(model.parameters(),lr= 2e-5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/rbt3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [57]:
def evaluate():
    model.eval()
    acc_num = 0
    with torch.inference_mode():
        for batch in validloader:
            output = model(**batch)
            pred = torch.argmax(output.logits, dim = -1)
            acc_num += (pred.long() == batch["labels"].long()).float().sum()
    return acc_num / len(validset)

def train(epoch = 3 , log_step = 100):
    global_step = 0
    for ep in range(epoch):
        model.train()
        for batch in trainloader:
            optimizer.zero_grad()
            output = model(**batch)
            output.loss.backward()
            optimizer.step()
            if global_step % log_step == 0 :
                print(f" epoch : {ep} , global_step: {global_step}, loss: {output.loss.item()}")
            global_step += 1
        acc = evaluate()
        print(f"epoch:{ep} , accuracy:{acc}")

In [58]:
train()

 epoch : 0 , global_step: 0, loss: 0.7863261103630066
 epoch : 0 , global_step: 100, loss: 0.2603783905506134
 epoch : 0 , global_step: 200, loss: 0.32615378499031067
epoch:0 , accuracy:0.8969072103500366
 epoch : 1 , global_step: 300, loss: 0.23437638580799103
 epoch : 1 , global_step: 400, loss: 0.21342067420482635
epoch:1 , accuracy:0.8956185579299927
 epoch : 2 , global_step: 500, loss: 0.23003661632537842
 epoch : 2 , global_step: 600, loss: 0.21387532353401184
epoch:2 , accuracy:0.8917526006698608


In [59]:
sen = "这家店不错！"
model.eval()
id2label = {0: "差评" , 1: "好评"}

with torch.inference_mode():
    inputs = tokenizer(sen , return_tensors= "pt")
    logits = model(**inputs).logits
    pred = torch.argmax(logits , dim = -1)
    print(f"输入 : {sen} \n 输出 : {id2label.get(pred.item())}")

输入 : 这家店不错！ 
 输出 : 好评
